In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [2]:
seed_value = 20
print("Train with random seed: ", seed_value)

import warnings

warnings.filterwarnings('ignore')

import os

os.environ['PYTHONHASHSEED'] = str(seed_value)
import random

random.seed(seed_value)
import numpy as np

np.random.seed(seed_value)
import tensorflow as tf

tf.random.set_seed(seed_value)
from keras import backend as K

Train with random seed:  20


In [3]:
# 위노그라드 알고리즘 설정
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

# 데이터 로드

In [4]:
rootPath = './vision/'

In [5]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [6]:
imageGenerator = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    preprocessing_function = get_random_eraser(v_l=0, v_h=255),
    validation_split=.1
)

In [7]:
trainGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='training'
)

Found 1157 images belonging to 3 classes.


In [8]:
validationGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='validation'
)

Found 127 images belonging to 3 classes.


# 모델 구성

In [9]:
model = Sequential()
model.add(ResNet50(include_top=True, weights=None, input_shape=(100, 100, 3), classes=3))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 3)                 23593859  
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_________________________________________________________________


In [10]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['acc'],
)

In [11]:
# epochs = 150
epochs = 5
history = model.fit_generator(
    trainGen, 
    epochs=epochs,
#     steps_per_epoch=trainGen.samples / epochs, 
    validation_data=validationGen,
#     validation_steps=trainGen.samples / epochs,
)

Epoch 1/5
37/37 [==============================] - 51s 1s/step - loss: 0.4777 - acc: 0.7494 - val_loss: 2847.7788 - val_acc: 0.6535
Epoch 2/5
37/37 [==============================] - 47s 1s/step - loss: 0.2342 - acc: 0.8574 - val_loss: 9.8491 - val_acc: 0.7165
Epoch 3/5
37/37 [==============================] - 48s 1s/step - loss: 0.2149 - acc: 0.8816 - val_loss: 0.2822 - val_acc: 0.8425
Epoch 4/5
37/37 [==============================] - 48s 1s/step - loss: 0.1953 - acc: 0.8911 - val_loss: 0.2803 - val_acc: 0.8268
Epoch 5/5
37/37 [==============================] - 48s 1s/step - loss: 0.1898 - acc: 0.8773 - val_loss: 0.2837 - val_acc: 0.8110


# 평가

In [12]:
testGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'test_set'),
    target_size=(100, 100),
)

Found 15 images belonging to 3 classes.


In [13]:
pred = model.predict(testGen)

In [14]:
img_list = testGen.filenames

In [15]:
pred_list = []
for i in range(len(pred)):
    pred_list.append(np.argmax(pred[i]))

In [16]:
df = pd.DataFrame(list(zip(img_list, pred_list)), columns =['Name', 'pred'])

In [17]:
df.to_csv('submission.csv', index=False)

In [18]:
print("정확도 :", model.evaluate_generator(testGen))

정확도 : [0.41317227482795715, 0.6666666865348816]


In [19]:
true_y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

In [20]:
(np.array(pred_list) == true_y).sum() / len(true_y)

0.5333333333333333

- 1: 0.3333333333333333
- 2: 0.3333333333333333
- 3: 0.3333333333333333
- 4: 0.4
- 5: 0.26666666666666666
- 6: 0.5333333333333333

### seed
0: 0.26666666666666666
10: 0.26666666666666666